In [1]:
pip install -U googlemaps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install googletrans==4.0.0-rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import tensorflow as tf
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np

In [4]:
hotel = pd.read_json("./hotel.json")
restaurant = pd.read_json("./restaurant.json")
todo = pd.read_json("./todo.json")

In [14]:
def recommend_places(activity, category, latitude, longitude, quantity):
    # Get the user's preferences from the request

    # Define the user's preferences
    activity = str(activity)
    category = str(category)
    latitude = latitude
    longitude = longitude

    # Calculate the distance between the user's location and the places' locations
    todo["distance"] = tf.sqrt(
        tf.square(todo["latitude"] - latitude) + tf.square(todo["longitude"] - longitude)
    )

    # Filter the places based on the user's preferences
    filtered_places = todo[
        (todo["activity"] == activity) & (todo["category"] == category)
    ]

    # Check if there are any matching places
    if filtered_places.empty:
        return {"recommendations": []}

    # Perform collaborative filtering using k-nearest neighbors
    X = filtered_places[["latitude", "longitude", "rating", "distance"]].values
    nbrs = NearestNeighbors(n_neighbors=quantity).fit(X)
    distances, indices = nbrs.kneighbors([[latitude, longitude, 5, 0]])

    # Get the top recommendations based on nearest neighbors
    top_indices = indices[0]
    top_recommendations = filtered_places.iloc[top_indices]

    # Create a response dictionary with the recommendations
    recommendations = []
    for _, row in top_recommendations.iterrows():
        recommendation = row.to_dict()
        recommendations.append(recommendation)

    return {"place_recommendation": recommendations}

In [7]:
import googlemaps
import requests
import json

# Replace 'YOUR_API_KEY' with your actual API key
api_key = ''

# Create a client instance
gmaps = googlemaps.Client(api_key)

def get_traffic_condition(latitude_origin, longitude_origin, latitude_destination, longitude_destination):
  # Request traffic information
  traffic_result = gmaps.directions((latitude_origin, longitude_origin), (latitude_destination, longitude_destination), mode="driving", departure_time="now", traffic_model="best_guess")

  if traffic_result == []:
    return {}
  # Extract the traffic information
  duration_without_traffic = traffic_result[0]['legs'][0]['duration']['value']
  duration_in_traffic = traffic_result[0]['legs'][0]['duration_in_traffic']['value']
  traffic_ratio = duration_in_traffic / duration_without_traffic
  if traffic_ratio <= 0.7:  # Current if the ratio of travel time to traffic <= 0.7
    traffic_status = "Lancar"
  elif traffic_ratio <= 1.3:  # Crowded if the ratio of travel time to traffic is between 0.7 and 1.3
    traffic_status = "Ramai"
  elif traffic_ratio <= 2.0:  # Padat jika rasio waktu tempuh dengan lalu lintas antara 1.3 dan 2.0
    traffic_status = "Padat"
  else:  # Macet jika rasio waktu tempuh dengan lalu lintas > 2.0
    traffic_status = "Macet"

  return {
      "duration_without_traffic":duration_without_traffic,
      "duration_with_traffic":duration_in_traffic,
      "traffic_ratio":traffic_ratio,
      "traffic_status":traffic_status
  }

In [6]:
import googlemaps
import requests
import json
import re
from googletrans import Translator


# Replace 'YOUR_API_KEY' with your actual API key
api_key = ''

# Create a client instance
gmaps = googlemaps.Client(api_key)

# Define the latitude and longitude coordinates
latitude_origin = -1.2654
longitude_origin = 116.8311
latitude_destination = -1.2654
longitude_destination = 116.9311

def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def capitalize_first_letter(sentence):
    words = sentence.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)

def create_route_description(latitude_origin, longitude_origin, latitude_destination, longitude_destination):
  # Request traffic information
  traffic_result = gmaps.directions((latitude_origin, longitude_origin), (latitude_destination, longitude_destination), mode="driving", departure_time="now", traffic_model="best_guess")
  if traffic_result == []:
    return {}

  steps = traffic_result[0]['legs'][0]['steps']

  translator = Translator()

  description = ''

  for i, step in enumerate(steps):
    instruction = remove_html_tags(step['html_instructions'])
    distance = step['distance']['text']
    duration = step['duration']['text']

    if i == 0:
      description += f"Mulailah dengan {instruction.lower()} sejauh {distance} selama {duration}.\n"
    elif i == len(steps) - 1:
      description += f"Beloklah ke {instruction.lower()}.\nTujuan akhir akan berada di sebelah kiri."
    else:
      description += f"{capitalize_first_letter(instruction)}. Jarak yang ditempuh sejauh {distance} selama {duration}.\n"

  description = translator.translate(description, dest='id', src='en').text

  return description

In [8]:
get_traffic_condition(-1.273225,116.83338, -1.273225,116.833898)

{'duration_without_traffic': 191,
 'duration_with_traffic': 200,
 'traffic_ratio': 1.0471204188481675,
 'traffic_status': 'Ramai'}

In [9]:
latitude_origin = -1.2654
longitude_origin = 116.8311
latitude_destination = -1.2654
longitude_destination = 116.9311

print(create_route_description(latitude_origin, longitude_origin, latitude_destination, longitude_destination))

Mulailah Delan Head Southeast Sejauh 8 M Selama 1 mnt.
Belok kiri menuju JL.Telaga Sari II.Jarak Yang Ditempuh SEJAUH 28 M Selama 1 mnt.
Belok kanan ke JL.Telaga Sari II.Jarak Yang Ditempuh SEJAUH 0,4 km Selama 2 menit.
Belok kiri ke JL.Telaga Sari.Jarak Yang Ditempuh SEJAUH 0,2 km Selama 1 menit.
Belok kanan ke JL.Re martadinata.Jarak Yang Ditempuh SEJAUH 0,2 km Selama 1 menit.
Belok kiri ke JL.Kapten Piere Tendean.Jarak Yang Ditempuh SEJAUH 0,3 km Selama 1 menit.
Belok kanan setelah Ikan Hias (di sebelah kiri) melewati Salon Widya (di sebelah kiri dalam 650m).Jarak Yang Ditempuh SEJAUH 1.2 km Selama 3 menit.
Belok kiri setelah LPK Kaltim Balikpapan (di sebelah kiri) melewati KFC Coffee Pasar Baru (di sebelah kiri).Jarak Yang Ditempuh SEJAUH 2.2 km Selama 5 menit.
Sedikit hak untuk tetap di JL.Jenderal Sudirman.Jarak Yang Ditempuh SEJAUH 2.5 km Selama 5 menit.
Lanjutkan ke JL.Marsma R. Iswahyudi.Jarak Yang Ditempuh SEJAUH 4.0 km Selama 7 menit.
Lanjutkan langsung untuk tetap di JL.Mar

In [150]:
def recommend_places_with_traffic(activity, category, latitude, longitude, quantity):
    # Get the user's preferences from the request

    # Define the user's preferences
    activity = str(activity)
    category = str(category)
    latitude = latitude
    longitude = longitude

    # Calculate the distance between the user's location and the places' locations
    todo["distance"] = tf.sqrt(
        tf.square(todo["latitude"] - latitude) + tf.square(todo["longitude"] - longitude)
    )

    for index, row in todo.iterrows():
        # Lakukan operasi lain yang diperlukan pada setiap baris

        traffic_condition = get_traffic_condition(latitude, longitude, row['latitude'], row['longitude'])
        if traffic_condition:
            for key, value in traffic_condition.items():
                todo.at[index, key] = value

    # Filter the places based on the user's preferences
    filtered_places = todo[
        (todo["activity"] == activity) & (todo["category"] == category)
    ]

    # Check if there are any matching places
    if filtered_places.empty:
        return {"recommendations": []}

    # Perform collaborative filtering using k-nearest neighbors
    X = filtered_places[["latitude", "longitude", "rating", "distance", "traffic_ratio"]].values
    nbrs = NearestNeighbors(n_neighbors=quantity).fit(X)
    distances, indices = nbrs.kneighbors([[latitude, longitude, 5, 0, 0]])

    # Get the top recommendations based on nearest neighbors
    top_indices = indices[0]
    top_recommendations = filtered_places.iloc[top_indices]

    # Create a response dictionary with the recommendations
    recommendations = []
    for _, row in top_recommendations.iterrows():
        recommendation = row.to_dict()
        recommendations.append(recommendation)

    return {"place_recommendation": recommendations}

In [ ]:
recommend_places_with_traffic('Outdoor','Beach',-1.273225,116.83338, 2)

In [ ]:
recommend_places('Outdoor','Beach',-1.273225,116.83338, 2)

In [157]:
# Function to recommend restaurants based on latitude and longitude
def recommend_restaurants(latitude, longitude, quantity):

     # Calculate the distance between the user's location and the places' locations
     restaurant["distance"] = tf.sqrt(
         tf.square(restaurant["latitude"] - latitude) + tf.square(restaurant["longitude"] - longitude)
     )

     # Takes latitude, longitude, rating, and distance columns as features
     X = restaurant[['latitude', 'longitude', 'rating', 'distance']].values

     # Build the KNN model with the number of neighbors (k) = quantity
     knn = NearestNeighbors(n_neighbors=quantity)
     knn.fit(X)

     # Search for 5 nearest neighbors of given position
     distances, indices = knn.kneighbors([[latitude, longitude, 5, 0]])

     # Displays recommended restaurants
     restaurant_recommendations = []
     for index in indices[0]:
         recommendation = restaurant.iloc[index].to_dict()
         restaurant_recommendations. append(recommendation)

     # Checks if distance is a multiple of 20, then sorts by best rating
     if int(restaurant.iloc[indices[0][0]]['distance']) % 20 == 0:
         restaurant_recommendations = sorted(restaurant_recommendations, key=lambda x: x['rating'], reverse=True)

     return {"restaurant_recommendation": restaurant_recommendations}

In [156]:
# Function to recommend restaurants based on latitude and longitude
def recommend_hotels(latitude, longitude, quantity):

     # Calculate the distance between the user's location and the places' locations
     hotel["distance"] = tf.sqrt(
         tf.square(hotel["latitude"] - latitude) + tf.square(hotel["longitude"] - longitude)
     )

     # Takes latitude, longitude, rating, and distance columns as features
     X = hotel[['latitude', 'longitude', 'hotel_class', 'rating', 'distance']].values

     # Build the KNN model with the number of neighbors (k) = quantity
     knn = NearestNeighbors(n_neighbors=quantity)
     knn.fit(X)

     # Search for 5 nearest neighbors of given position
     distances, indices = knn.kneighbors([[latitude, longitude, 5, 5, 0]])

     # Displays recommended restaurants
     recommendations = []
     for index in indices[0]:
         recommendation = hotel.iloc[index].to_dict()
         recommendations. append(recommendation)

     # Checks if distance is a multiple of 20, then sorts by best rating
     if int(hotel.iloc[indices[0][0]]['distance']) % 20 == 0:
         recommendations = sorted(recommendations, key=lambda x: x['rating'], reverse=True)

     return {"hotel_recommendation": recommendations}

In [ ]:
def itinerary(activity, category, latitude, longitude):
  todo_recommendation = recommend_places(activity, category, latitude, longitude, 1)

  if len(todo_recommendation) > 0:
      todo_latitude = todo_recommendation['place_recommendation'][0]['latitude']
      todo_longitude = todo_recommendation['place_recommendation'][0]['longitude']

      hotel_recommendation = recommend_hotels(todo_latitude, todo_longitude, 3)
      restaurant_recommendation = recommend_restaurants(todo_latitude, todo_longitude, 3)

      return {
          'todo_recommendation': todo_recommendation,
          'hotel_recommendation': hotel_recommendation,
          'restaurant_recommendation': restaurant_recommendation
      }

  else:
      return {
          'todo_recommendation':[],
          'hotel_recommendation':[],
          'restaurant_recommendation':[]
      }



In [152]:
def itinerary_with_traffic(activity, category, latitude, longitude):
  todo_recommendation = recommend_places_with_traffic(activity, category, latitude, longitude, 1)

  if len(todo_recommendation) > 0:
      todo_latitude = todo_recommendation['place_recommendation'][0]['latitude']
      todo_longitude = todo_recommendation['place_recommendation'][0]['longitude']

      hotel_recommendation = recommend_hotels(todo_latitude, todo_longitude, 3)
      restaurant_recommendation = recommend_restaurants(todo_latitude, todo_longitude, 3)

      return {
          'todo_recommendation': todo_recommendation,
          'hotel_recommendation': hotel_recommendation,
          'restaurant_recommendation': restaurant_recommendation
      }

  else:
      return {
          'todo_recommendation':[],
          'hotel_recommendation':[],
          'restaurant_recommendation':[]
      }



In [154]:
# Example usage
user_activity = "Outdoor"
user_category = "Beach"
user_latitude = -1.2654
user_longitude = 116.8311

In [ ]:
itinerary(user_activity, user_category, user_latitude, user_longitude)

In [ ]:
itinerary_with_traffic(user_activity, user_category, user_latitude, user_longitude)

In [15]:
def recommend_places_activity(activity, latitude, longitude, quantity):
    # Get the user's preferences from the request

    # Define the user's preferences
    activity = str(activity)
    latitude = latitude
    longitude = longitude

    # Calculate the distance between the user's location and the places' locations
    todo["distance"] = tf.sqrt(
        tf.square(todo["latitude"] - latitude) + tf.square(todo["longitude"] - longitude)
    )

    # Filter the places based on the user's preferences
    filtered_places = todo[
        (todo["activity"] == activity)
    ]

    # Check if there are any matching places
    if filtered_places.empty:
        return {"recommendations": []}

    # Perform collaborative filtering using k-nearest neighbors
    X = filtered_places[["latitude", "longitude", "rating", "distance"]].values
    nbrs = NearestNeighbors(n_neighbors=quantity).fit(X)
    distances, indices = nbrs.kneighbors([[latitude, longitude, 5, 0]])

    # Get the top recommendations based on nearest neighbors
    top_indices = indices[0]
    top_recommendations = filtered_places.iloc[top_indices]

    # Create a response dictionary with the recommendations
    recommendations = []
    for _, row in top_recommendations.iterrows():
        recommendation = row.to_dict()
        recommendations.append(recommendation)

    return {"place_recommendation": recommendations}

In [ ]:
recommend_places_activity('Outdoor', user_latitude, user_longitude, 1)

In [12]:
from datetime import datetime, timedelta
import pandas as pd

def itinerary_by_date(activity, latitude, longitude, start_date, end_date, quantity_per_day):

    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    data = []
    data_route = []

    for index, date in enumerate(date_range):
        current_date = date.strftime('%Y-%m-%d')
        result = recommend_places_activity(activity, latitude, longitude, quantity_per_day)['place_recommendation']
        data.append({current_date: {
            'date': current_date,
            'todo': result
        }})
        data_route.append(result)


    return data

In [16]:
user_activity = "Outdoor"
user_latitude = -1.2654
user_longitude = 116.11
start_date = '2023-06-15'
end_date = '2023-06-16'
quantity_per_day = 2

itinerary_by_date(user_activity, user_latitude, user_longitude, start_date, end_date, quantity_per_day)

[{'2023-06-15': {'date': '2023-06-15',
   'todo': [{'location_id': 10821404,
     'place_name': 'Demonstration Monument',
     'city': 'Balikpapan',
     'rating': 4.5,
     'latitude': -1.24984,
     'longitude': 116.82822,
     'opening_hours': {'week_ranges': [[{'open_time': 0, 'close_time': 2400}],
       [{'open_time': 0, 'close_time': 2400}],
       [{'open_time': 0, 'close_time': 2400}],
       [{'open_time': 0, 'close_time': 2400}],
       [{'open_time': 0, 'close_time': 2400}],
       [{'open_time': 0, 'close_time': 2400}],
       [{'open_time': 0, 'close_time': 2400}]],
      'timezone': 'Asia/Makassar'},
     'contact_number': None,
     'photo': None,
     'address': 'Jl. Minyak, Balikpapan 76123 Indonesia',
     'min_price': 0,
     'max_price': 0,
     'activity': 'Outdoor',
     'category': 'Culture',
     'description': 'Monumen di Balikapapan',
     'distance': 0.7183885313672563},
    {'location_id': 14080808,
     'place_name': 'Aquaboom Waterpark',
     'city': 'Bal